In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from data_import import (
    load_data, load_ecb_1y_yield,
    fill_liabilities, drop_high_leverage_firms,
    prepare_nig_inputs
)
import numpy as np
from dataclasses import dataclass
from scipy.optimize import brentq
from scipy.integrate import quad

print(Path.cwd())

c:\Users\vkeenan\OneDrive - Delft University of Technology\Documents\University\QRM\Accenture Project\code


In [5]:
# data import and input panel preparation
ret_daily, bs, coverage = load_data(
    Path.cwd() / "data/raw/Jan2025_Accenture_Dataset_ErasmusCase.xlsx",
    start_date="2012-01-01",
    end_date="2025-12-19",
    enforce_coverage=True,
    coverage_tol=0.95,
    liabilities_scale="auto",
    verbose=True,
)

df_rf = load_ecb_1y_yield(
    startPeriod="2010-01-01",
    endPeriod="2025-12-31",
    out_file="ecb_yc_1y_aaa.xml",
    verify_ssl=True,  # recommended if it works
)

df_cal = ret_daily[["date"]].drop_duplicates().sort_values("date").reset_index(drop=True)

debt_daily = fill_liabilities(bs, df_cal)

ret_filt, bs_filt, lev_by_firm, dropped = drop_high_leverage_firms(
    ret_daily,
    bs,
    df_calendar=df_cal,
    debt_daily=debt_daily,
    lev_threshold=8.0,
    lev_agg="median",
    verbose=True,
)

# keep debt panel consistent with filtered firms
keep = set(ret_filt["gvkey"].astype(str).unique())
debt_daily_filt = debt_daily[debt_daily["gvkey"].astype(str).isin(keep)].copy()

nig_df, em_cache = prepare_nig_inputs(ret_filt, bs_filt, df_rf, debt_daily=debt_daily_filt, build_em=False)
print(nig_df.head())
print(nig_df.shape)
print(nig_df.describe())

[load_data] Firms (ret_daily): 46
[load_data] Date range (ret_daily): 2012-01-03 .. 2025-12-19
[load_data] Coverage min/median/max: 0.999 / 1.000 / 1.000
[load_data] liabilities_scale_used: 1e+06
[load_data] QA mcap_reported<=0 rows (raw windowed mkt): 62
Data has been written to ecb_yc_1y_aaa.xml
[drop_high_leverage_firms] agg=median, threshold=8.0
[drop_high_leverage_firms] firms before: 46 | after: 36
[drop_high_leverage_firms] dropped firms: 10
    gvkey       date             E          isin  \
0  100022 2012-01-03  3.328431e+10  DE0005190003   
1  100080 2012-01-03  4.268705e+10  DE000BAY0017   
2  100312 2012-01-03  1.469717e+09  DE0007030009   
3  100581 2012-01-03  4.935351e+10  FR0000120321   
4  100957 2012-01-03  2.931851e+10  ES0144580Y14   

                        company country_iso         r             L  
0  BAYERISCHE MOTOREN WERKE AKT         DEU  0.001177  8.576700e+10  
1                      BAYER AG         DEU  0.001177  3.254300e+10  
2                RHEINME

### Goal

For each firm/date, solve for \(A_t\) in:

$$
E_t = C_{\text{NIG}}(A_t, L_t, r_t, \tau; \vartheta_A),
\qquad
\tau = T - t
$$

where \(C_{\text{NIG}}\) is the NIG‑based equity‑as‑call valuation used by Ahčan & Jovan (Eq. 26).

### Outputs

**Daily asset values**

$$
\hat{A}_t
$$

**Asset log returns**

$$
\Delta \ln \hat{A}_t
$$

#### NIG Components

**NIG log‑mgf**

$$
\kappa(u)
$$

**Esscher parameter \(\theta\)**  
Solving the martingale condition (risk‑neutralization)

$$
\theta \text{ such that the RN drift condition holds}
$$

**NIG call price**

$$
C_{\text{NIG}}(A, L, r, \tau)
$$

via a robust Fourier probability representation.

**Asset inversion**

$$
E \;\longmapsto\; \hat{A}
$$

via Brent root finding (same spirit as the paper’s “uniroot” step).



In [6]:
@dataclass(frozen=True)
class NIGParams:
    # NIG for log-returns Z: (alpha, beta, delta, mu) under P, per unit time
    alpha: float
    beta: float
    delta: float
    mu: float

    def validate(self):
        if not (self.alpha > abs(self.beta)):
            raise ValueError("Need alpha > |beta| for NIG.")
        if not (self.delta > 0):
            raise ValueError("Need delta > 0 for NIG.")

In [7]:
def nig_kappa(u, p: NIGParams, tau: float):
    """
    Vectorized log-mgf kappa(u; tau) for NIG increments.
    u can be scalar or numpy array (real/complex).
    """
    p.validate()
    a, b, d, m = p.alpha, p.beta, p.delta, p.mu

    u = np.asarray(u, dtype=np.complex128)

    term0 = np.sqrt(a*a - b*b)
    term1 = np.sqrt(a*a - (b + u)*(b + u))
    return (m * u + d * (term0 - term1)) * tau


def solve_esscher_theta(p: NIGParams, r: float, tau: float, *, bracket=(-10.0, 10.0)) -> float:
    """
    Solve for theta in:
      kappa(theta+1) - kappa(theta) = r*tau
    """
    lo, hi = bracket

    def g(th):
        # th is real
        return (nig_kappa(th + 1.0, p, tau) - nig_kappa(th, p, tau)).real - (r * tau)

    # ensure bracket contains root; expand if needed
    f_lo, f_hi = g(lo), g(hi)
    tries = 0
    while f_lo * f_hi > 0 and tries < 10:
        lo *= 2
        hi *= 2
        f_lo, f_hi = g(lo), g(hi)
        tries += 1

    if f_lo * f_hi > 0:
        raise RuntimeError("Could not bracket theta root; try different initial NIG params or bracket.")

    return float(brentq(g, lo, hi, maxiter=200))


def cf_logA_T_vec(u, lnA: float, p: NIGParams, theta: float, tau: float):
    """
    Vectorized characteristic function of ln A_T under Esscher tilt theta.
    u can be scalar or numpy array (real/complex).
    """
    u = np.asarray(u, dtype=np.complex128)
    iu = 1j * u
    psi = nig_kappa(theta + iu, p, tau) - nig_kappa(theta, p, tau)
    return np.exp(1j * u * lnA + psi)


def _Pj_prob(j, A, K, p, theta, tau, *, U=120.0, n=8000) -> float:
    if j not in (1, 2):
        raise ValueError("j must be 1 or 2")
    if A <= 0 or K <= 0 or tau <= 0:
        return np.nan

    lnA = float(np.log(A))
    lnK = float(np.log(K))

    u = np.linspace(1e-8, U, n)

    shift = -1j * (j - 1)
    phi = cf_logA_T_vec(u + shift, lnA, p, theta, tau)
    phi_shift = cf_logA_T_vec(shift, lnA, p, theta, tau)

    expo = np.exp(-1j * u * lnK)
    q = expo * phi / (1j * u * phi_shift)

    integral = np.trapezoid(np.real(q), u)
    Pj = 0.5 + (1.0 / np.pi) * integral
    return float(np.clip(Pj, 0.0, 1.0))


def call_nig_with_theta(A, K, r, tau, p, theta, *, U=120.0, n=8000) -> float:
    P1 = _Pj_prob(1, A, K, p, theta, tau, U=U, n=n)
    P2 = _Pj_prob(2, A, K, p, theta, tau, U=U, n=n)

    C = A * P1 - K * np.exp(-r * tau) * P2

    lower = max(A - K * np.exp(-r * tau), 0.0)
    upper = A

    if not np.isfinite(C):
        return np.nan
    return float(min(upper, max(lower, C)))


def invert_asset_one_date(E_obs, L, r, tau, p, *, A_prev=None, U=120.0, n=8000, bracket_mult=5.0):
    theta = solve_esscher_theta(p, r, tau)

    A0 = max(E_obs + 1e-12, E_obs + L, (A_prev if A_prev is not None else 0.0))
    A_lo = max(1e-12, 0.1 * L)
    A_hi = max(A0, E_obs + bracket_mult * L)

    def f(A):
        return call_nig_with_theta(A, L, r, tau, p, theta, U=U, n=n) - E_obs

    f_lo, f_hi = f(A_lo), f(A_hi)
    tries = 0
    while f_lo * f_hi > 0 and tries < 20:
        A_hi *= 2.0
        f_hi = f(A_hi)
        tries += 1

    if f_lo * f_hi > 0:
        raise RuntimeError("Could not bracket root for A.")

    A_hat = float(brentq(f, A_lo, A_hi, maxiter=200))
    return A_hat, theta

In [8]:
# Pick BMW
gv = "100022"
g = nig_df[nig_df["gvkey"].astype(str) == gv].sort_values("date").copy()

# pick one month-end date (last date of a month available in your trading calendar)
g["month"] = g["date"].dt.to_period("M")
month_ends = g.groupby("month")["date"].max().sort_values()
asof = month_ends.iloc[0]  # earliest month-end in sample (change as you like)

row = g[g["date"] == asof].iloc[0]
E_obs, L, r = float(row["E"]), float(row["L"]), float(row["r"])
tau = 1.0  # 1Y horizon in years (you can later use ACT/365 convention, but 1.0 is fine for Milestone 1)

# Starter NIG parameters (placeholder, just for Milestone 1 stability)
# Must satisfy alpha > |beta| and delta > 0.
p0 = NIGParams(alpha=15.0, beta=-3.0, delta=0.20, mu=0.00)

A_hat, theta = invert_asset_one_date(E_obs, L, r, tau, p0, U=150.0)
print("Date:", asof.date())
print("E_obs:", E_obs)
print("L:", L)
print("r:", r)
print("A_hat:", A_hat)
print(call_nig_with_theta(A_hat, L, r, tau, p0, theta))

Date: 2012-01-31
E_obs: 39364465866.44
L: 85767000000.0
r: 0.0020614851688972656
A_hat: 124954840585.7082
39364465866.44


We now have, for a single date:

$$
E_t \;\longrightarrow\; \hat{A}_t
$$

We will now extend this to a full time series:

$$
\{E_t\}_{t \in \text{month-ends}}
\;\longrightarrow\;
\{\hat{A}_t\}_{t \in \text{month-ends}}
\;\longrightarrow\;
\Delta \ln \hat{A}_t
$$

#### Outputs

- **Monthly asset levels**

$$
\{\hat{A}_t\}_{t \in \text{month-ends}}
$$

- **Monthly asset log‑returns**

$$
\Delta \ln \hat{A}_t
$$


In [20]:
def invert_assets_monthly_for_firm(
    g: pd.DataFrame,
    p: NIGParams,
    tau: float = 1.0,
    U: float = 150.0,
):
    g = g.sort_values("date").copy()
    g["month"] = g["date"].dt.to_period("M")
    month_ends = g.groupby("month")["date"].max().sort_values()

    results = []
    A_prev = None

    for d in month_ends:
        row = g[g["date"] == d].iloc[0]
        E_obs = float(row["E"])
        L     = float(row["L"])
        r     = float(row["r"])

        # unpack two outputs
        A_hat, theta = invert_asset_one_date(
            E_obs, L, r, tau, p,
            A_prev=A_prev,
            U=U
        )

        results.append((d, E_obs, L, r, A_hat, theta))
        A_prev = A_hat

    out = pd.DataFrame(results, columns=["date", "E", "L", "r", "A_hat", "theta"])
    out["logA"] = np.log(out["A_hat"])
    out["logret_A"] = out["logA"].diff()
    return out

In [9]:
gv = "100022"
g = nig_df[nig_df["gvkey"].astype(str) == gv].copy()

p0 = NIGParams(alpha=15.0, beta=-3.0, delta=0.20, mu=0.00)

import time
start = time.time()

bmw_assets = invert_assets_monthly_for_firm(g, p0, U=120.0)

print("Runtime:", time.time() - start)
print(bmw_assets.head())

NameError: name 'invert_assets_monthly_for_firm' is not defined

Keep monthly as baseline, but add extra inversion dates during stress  
(without going daily).  
This updates the mapping

$$
E \;\longmapsto\; A
$$

more frequently when equity dynamics suggest the latent asset process may have shifted.

##### Trigger Design

Use equity log‑returns (already in your panel as `logret_mcap`) and define triggers when:

#### **Large absolute daily move**

$$
|\Delta \log E_t| \;>\; q_{0.99}\big(|\Delta \log E|\big)
$$

where the quantile is computed **rolling over the last 252 trading days**, firm‑specific.

#### Inversion Calendar

- all **month‑end** trading dates  
- **plus all trigger dates**  
- *(optional)* plus the next **1–3 days** after a trigger to capture aftershocks


In [12]:
def build_inversion_calendar(
    g_panel: pd.DataFrame,          # nig_df slice for firm (has date, E, L, r)
    g_ret: pd.DataFrame,            # ret_filt slice for firm (has date, logret_mcap)
    *,
    ret_col: str = "logret_mcap",
    roll_window: int = 252,
    q: float = 0.99,
    add_post_days: int = 2,
):
    g_panel = g_panel.sort_values("date").copy()
    g_ret = g_ret.sort_values("date").copy()

    # month-end trading dates from the panel calendar
    month_end = g_panel.groupby(g_panel["date"].dt.to_period("M"))["date"].max()

    # rolling quantile threshold for |returns|
    absr = g_ret[ret_col].abs()
    thr = absr.rolling(roll_window, min_periods=roll_window // 2).quantile(q)

    trigger_dates = g_ret.loc[absr > thr, "date"]

    # add post-trigger days based on the panel’s trading-date index (safer)
    if add_post_days > 0:
        dates = g_panel["date"].to_numpy()
        date_to_pos = {d: i for i, d in enumerate(dates)}
        extra = []
        for d in trigger_dates:
            i = date_to_pos.get(d, None)
            if i is None:
                continue
            for k in range(1, add_post_days + 1):
                if i + k < len(dates):
                    extra.append(dates[i + k])
        trigger_dates = pd.to_datetime(pd.Index(trigger_dates.tolist() + extra)).drop_duplicates()

    cal = (
        pd.to_datetime(pd.Index(month_end.tolist()).append(pd.Index(trigger_dates)))
        .drop_duplicates()
        .sort_values()
    )
    return pd.DatetimeIndex(cal)


def invert_assets_on_calendar_for_firm(
    g: pd.DataFrame,
    p: NIGParams,
    dates: pd.DatetimeIndex,
    *,
    tau: float = 1.0,
    U: float = 120.0,
    n: int = 8000,
):
    g = g.sort_values("date").copy()
    g = g.set_index("date")

    results = []
    A_prev = None

    for d in dates:
        if d not in g.index:
            continue
        row = g.loc[d]
        E_obs = float(row["E"])
        L     = float(row["L"])
        r     = float(row["r"])

        A_hat, theta = invert_asset_one_date(
            E_obs, L, r, tau, p,
            A_prev=A_prev,
            U=U, n=n
        )
        results.append((d, E_obs, L, r, A_hat, theta))
        A_prev = A_hat

    out = pd.DataFrame(results, columns=["date", "E", "L", "r", "A_hat", "theta"]).sort_values("date")
    out["logA"] = np.log(out["A_hat"])
    out["logret_A"] = out["logA"].diff()
    return out


In [14]:
gv = "100022"

g_panel = nig_df[nig_df["gvkey"].astype(str) == gv].copy()
g_ret   = ret_filt[ret_filt["gvkey"].astype(str) == gv].copy()
g_ret = g_ret[["date", "logret_mcap"]].copy()
g_ret = g_panel[["date"]].merge(g_ret, on="date", how="left")

dates = build_inversion_calendar(g_panel, g_ret, q=0.99, add_post_days=2)
bmw_assets_trig = invert_assets_on_calendar_for_firm(g_panel, p0, dates)

print(bmw_assets_trig.head())
print(bmw_assets_trig.describe())

abs_all = g_ret["logret_mcap"].abs()
abs_trig = g_ret[g_ret["date"].isin(dates)]["logret_mcap"].abs()
print(abs_all.quantile([0.5, 0.9, 0.99, 0.999]))
print(abs_trig.quantile([0.5, 0.9, 0.99, 0.999]))
print("share of trigger days:", len(abs_trig)/len(abs_all))

        date             E             L         r         A_hat     theta  \
0 2012-01-31  3.936447e+10  8.576700e+10  0.002061  1.249548e+11  2.654517   
1 2012-02-29  4.179653e+10  8.576700e+10  0.001470  1.274375e+11  2.610187   
2 2012-03-30  4.059254e+10  9.632600e+10  0.001602  1.367644e+11  2.620070   
3 2012-04-30  4.322928e+10  9.632600e+10  0.001019  1.394572e+11  2.576356   
4 2012-05-31  3.678191e+10  9.632600e+10  0.000512  1.330586e+11  2.538390   

        logA  logret_A  
0  25.551218       NaN  
1  25.570892  0.019674  
2  25.641525  0.070633  
3  25.661024  0.019498  
4  25.614055 -0.046968  
                      date             E             L           r  \
count                  300  3.000000e+02  3.000000e+02  300.000000   
mean   2019-01-03 06:00:00  4.857056e+10  1.366155e+11    0.002538   
min    2012-01-31 00:00:00  2.355908e+10  8.576700e+10   -0.009130   
25%    2015-05-11 18:00:00  4.319166e+10  1.173660e+11   -0.006637   
50%    2018-09-27 12:00:00  4.8

To improve computational efficiency while preserving statistical validity, we use an event-driven inversion calendar to update the implied asset level A^t more frequently during periods of large equity moves. However, because trigger dates oversample the tails of the equity return distribution (selection bias), we do not use the trigger-subsampled series to estimate the NIG parameters. Instead, NIG parameters are estimated on a fixed, regular return grid (e.g., weekly or daily rolling windows), and the trigger mechanism is used only as a state-update device for the equity-to-asset inversion step.

#### EM Input Requirements

What the EM‑input extractor expects:

**Equity series**

$$
\{E_t\}_{t = 1}^{505}
$$

(last 505 observations)

**Risk‑free rate series**

$$
\{r_t\}_{t = 1}^{505}
$$

(last 505 observations)

**Liability level (scalar)**

$$
L
$$

chosen over the window (i.e. last value).

#### Weekly EM Workflow

Define EM dates — for example:

- every **Friday**, or  
- every **5th trading day**.

For each EM date:

1. Call  
   ```
   make_em_inputs(nig_df, gvkey, end_date=em_date, window=505, ...)
   ```
2. Run your EM initializer / estimator (the function referenced in your docstring).
3. Store fitted parameters in a cache:
   ```
   { em_date : p_hat }
   ```

##### Using EM Parameters During Asset Inversion

When inverting assets on the inversion calendar, use the **most recent** parameter estimate:

$$
\hat{p}_{\text{EM date}} \quad \text{such that} \quad \text{EM date} \le \text{inversion date}
$$

#### Summary

- **EM schedule:** weekly, fixed  
- **Inversion schedule:** month‑ends + triggers  
- Both reuse your existing working components


In [23]:
def build_weekly_calendar(g_panel: pd.DataFrame) -> pd.DatetimeIndex:
    """
    Weekly inversion/EM grid = last trading day of each week
    (robust to holidays because it uses observed trading dates).
    """
    d = g_panel[["date"]].drop_duplicates().sort_values("date")
    week_end = d.groupby(d["date"].dt.to_period("W"))["date"].max()
    return pd.DatetimeIndex(week_end.sort_values())


def build_weekly_em_windows(
    assets_weekly: pd.DataFrame,
    *,
    window_weeks: int = 104,   # ~2 years weekly
    L_pick: str = "last",      # or "median"
):
    df = assets_weekly.sort_values("date").copy()
    df = df.dropna(subset=["dlogA", "r", "L"])

    em_inputs = {}
    for i in range(window_weeks, len(df) + 1):
        w = df.iloc[i-window_weeks:i]
        end_date = w["date"].iloc[-1]

        dlogA = w["dlogA"].to_numpy(float)
        r_arr = w["r"].to_numpy(float)
        if L_pick == "last":
            L_scalar = float(w["L"].iloc[-1])
        elif L_pick == "median":
            L_scalar = float(w["L"].median())
        else:
            raise ValueError("L_pick must be 'last' or 'median'")

        em_inputs[pd.to_datetime(end_date)] = (dlogA, L_scalar, r_arr)

    return em_inputs


gv = "100022"
g_panel = nig_df[nig_df["gvkey"].astype(str) == gv].copy()

p0 = NIGParams(alpha=15.0, beta=-3.0, delta=0.20, mu=0.00)

dates_weekly = build_weekly_calendar(g_panel)
bmw_assets_weekly = invert_assets_on_calendar_for_firm(g_panel, p0, dates_weekly)

bmw_assets_weekly["logA"] = np.log(bmw_assets_weekly["A_hat"])
bmw_assets_weekly["dlogA"] = bmw_assets_weekly["logA"].diff()

print(len(dates_weekly), "weekly points")
print(bmw_assets_weekly.describe())

729 weekly points
                      date             E             L           r  \
count                  729  7.290000e+02  7.290000e+02  729.000000   
mean   2018-12-28 00:00:00  4.833134e+10  1.373487e+11    0.002991   
min    2012-01-06 00:00:00  2.490454e+10  8.576700e+10   -0.009079   
25%    2015-07-03 00:00:00  4.347007e+10  1.173660e+11   -0.006755   
50%    2018-12-28 00:00:00  4.821982e+10  1.411720e+11   -0.002541   
75%    2022-06-24 00:00:00  5.307190e+10  1.556380e+11    0.002141   
max    2025-12-19 00:00:00  7.220932e+10  1.727290e+11    0.035827   
std                    NaN  7.755054e+09  2.489014e+10    0.013500   

              A_hat       theta        logA    logret_A       dlogA  
count  7.290000e+02  729.000000  729.000000  728.000000  728.000000  
mean   1.851776e+11    2.719850   25.934448    0.000855    0.000855  
min    1.190473e+11    1.820191   25.502787   -0.049308   -0.049308  
25%    1.699347e+11    1.993925   25.858680   -0.005845   -0.005845  
5

In [25]:
assert {"r","L","A_hat","dlogA","date"}.issubset(bmw_assets_weekly.columns)
em_inputs = build_weekly_em_windows(bmw_assets_weekly, window_weeks=104)
print("EM windows:", len(em_inputs))  # should be about 729-104 ≈ 625

EM windows: 625


In [34]:
import numpy as np
from dataclasses import dataclass
from scipy.optimize import brentq
from scipy.special import kve  # exponentially-scaled modified Bessel K

@dataclass
class NIGParams:
    alpha: float
    beta: float
    delta: float
    mu: float

    def validate(self):
        if not (self.alpha > abs(self.beta) and self.delta > 0):
            raise ValueError("Need alpha > |beta| and delta > 0.")

def _safe_ratio_kve(nu_num, nu_den, x):
    # ratio K_nu_num(x) / K_nu_den(x) using scaled kve; scaling cancels in ratio
    return kve(nu_num, x) / kve(nu_den, x)

def em_fit_nig(
    x: np.ndarray,
    p_start: NIGParams,
    *,
    max_iter: int = 200,
    tol: float = 1e-6,
    min_alpha_gap: float = 1e-6,
    verbose: bool = False,
) -> NIGParams:
    """
    EM for NIG on 1D returns array x (e.g., weekly dlogA).
    Uses GIG posterior moments with Bessel K ratios.
    """
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    if x.size < 10:
        raise ValueError("Too few observations for EM.")

    p = NIGParams(**vars(p_start))
    p.validate()

    n = x.size

    for it in range(max_iter):
        # current derived parameter
        gamma = np.sqrt(max(p.alpha*p.alpha - p.beta*p.beta, 1e-18))
        psi = gamma*gamma  # = alpha^2 - beta^2

        # E-step: Y|X is GIG(lambda=-1/2, chi=delta^2+(x-mu)^2, psi=gamma^2)
        chi = p.delta*p.delta + (x - p.mu)**2
        chi = np.maximum(chi, 1e-18)
        s = np.sqrt(chi * psi)
        s = np.maximum(s, 1e-12)

        lam = -0.5
        # Moments:
        # E[Y]   = (K_{lam+1}(s)/K_lam(s)) * sqrt(chi/psi)
        # E[1/Y] = (K_{lam-1}(s)/K_lam(s)) * sqrt(psi/chi)
        ratio_p1 = _safe_ratio_kve(lam + 1.0, lam, s)   # K_{+1/2}/K_{-1/2}
        ratio_m1 = _safe_ratio_kve(lam - 1.0, lam, s)   # K_{-3/2}/K_{-1/2}

        Ey  = ratio_p1 * np.sqrt(chi / psi)
        E1y = ratio_m1 * np.sqrt(psi / chi)

        # M-step for mu, beta (solve 2x2 linear system derived from normal equations)
        W  = np.sum(E1y)
        A  = np.sum(E1y * x)
        X  = np.sum(x)
        Y  = np.sum(Ey)

        denom = (n*n / Y) - W
        if abs(denom) < 1e-18:
            # fallback: tiny ridge
            denom = np.sign(denom) * 1e-18 if denom != 0 else 1e-18

        mu_new = (n * X / Y - A) / denom
        beta_new = (X - n * mu_new) / Y

        # M-step for delta, gamma (closed form)
        S1 = np.sum(E1y)
        # denominator must be positive
        den_delta = S1 - (n*n / Y)
        if den_delta <= 1e-18:
            den_delta = 1e-18

        delta_new = np.sqrt(n / den_delta)
        gamma_new = (n * delta_new) / Y

        # recover alpha from gamma and beta
        alpha_new = np.sqrt(beta_new*beta_new + gamma_new*gamma_new)

        # enforce alpha > |beta|
        if alpha_new <= abs(beta_new) + min_alpha_gap:
            alpha_new = abs(beta_new) + min_alpha_gap + 1e-6

        # convergence check (relative)
        vec_old = np.array([p.alpha, p.beta, p.delta, p.mu])
        vec_new = np.array([alpha_new, beta_new, delta_new, mu_new])
        rel = np.max(np.abs(vec_new - vec_old) / (np.abs(vec_old) + 1e-8))

        p = NIGParams(alpha=float(alpha_new), beta=float(beta_new),
                      delta=float(delta_new), mu=float(mu_new))

        if verbose:
            print(f"[EM] iter={it:03d} rel_change={rel:.2e} "
                  f"alpha={p.alpha:.3f} beta={p.beta:.3f} delta={p.delta:.3f} mu={p.mu:.5f}")

        if rel < tol:
            break

    p.validate()
    return p


def run_weekly_inversion_plus_em(
    g_panel: pd.DataFrame,
    *,
    p0: NIGParams,
    window_weeks: int = 104,
    em_max_iter: int = 80,
    em_tol: float = 1e-6,
    U: float = 120.0,
    n: int = 2000
):
    """
    Fully iterative weekly loop:
    - Invert A_hat on weekly dates using current params p_t
    - After enough history, EM-fit p_{t+1} on last window_weeks of dlogA
    Returns:
      assets_weekly (df with A_hat, theta, dlogA, p_used fields),
      p_cache (dict end_date->NIGParams)
    """
    dates = build_weekly_calendar(g_panel)
    g_panel = g_panel.sort_values("date").copy()
    g_week = g_panel[g_panel["date"].isin(dates)].copy().sort_values("date")

    rows = []
    p_cache = {}
    p_curr = p0
    A_prev = None

    # store dlogA history for EM
    dlogA_hist = []

    for i, (_, row) in enumerate(g_week.iterrows()):
        d = pd.to_datetime(row["date"])
        E_obs = float(row["E"])
        L = float(row["L"])
        r = float(row["r"])

        # 1) Invert assets for this week using current params
        A_hat, theta = invert_asset_one_date(
            E_obs, L, r, tau=1.0, p=p_curr,
            A_prev=A_prev, U=U, n=n
        )

        logA = np.log(A_hat)
        if i == 0:
            dlogA = np.nan
        else:
            dlogA = logA - rows[-1]["logA"]

        rows.append({
            "date": d, "E": E_obs, "L": L, "r": r,
            "A_hat": A_hat, "theta": theta,
            "logA": logA, "dlogA": dlogA,
            "p_alpha": p_curr.alpha, "p_beta": p_curr.beta,
            "p_delta": p_curr.delta, "p_mu": p_curr.mu,
        })

        A_prev = A_hat

        # 2) Update EM params once we have enough dlogA history
        if np.isfinite(dlogA):
            dlogA_hist.append(dlogA)

        if len(dlogA_hist) >= window_weeks:
            x = np.array(dlogA_hist[-window_weeks:], float)
            p_new = em_fit_nig(
                x, p_curr,
                max_iter=em_max_iter,
                tol=em_tol,
                verbose=False,
            )
            p_curr = p_new
            p_cache[d] = p_curr

    assets_weekly = pd.DataFrame(rows)
    return assets_weekly, p_cache

In [35]:
gv = "100022"
g_panel = nig_df[nig_df["gvkey"].astype(str) == gv].copy()

p0 = NIGParams(alpha=15.0, beta=-3.0, delta=0.20, mu=0.00)

assets_weekly_iter, p_cache_iter = run_weekly_inversion_plus_em(
    g_panel,
    p0=p0,
    window_weeks=104,
    em_max_iter=80,   # warm-start -> usually enough
    em_tol=1e-6,
    U=120.0,
    n=2000
)

print(assets_weekly_iter[["date","A_hat","dlogA","p_alpha","p_beta","p_delta","p_mu"]].head())
print("weeks:", len(assets_weekly_iter), "em-updates:", len(p_cache_iter))

        date         A_hat     dlogA  p_alpha  p_beta  p_delta  p_mu
0 2012-01-06  1.190473e+11       NaN     15.0    -3.0      0.2   0.0
1 2012-01-13  1.208258e+11  0.014829     15.0    -3.0      0.2   0.0
2 2012-01-20  1.238685e+11  0.024871     15.0    -3.0      0.2   0.0
3 2012-01-27  1.244749e+11  0.004884     15.0    -3.0      0.2   0.0
4 2012-02-03  1.277593e+11  0.026044     15.0    -3.0      0.2   0.0
weeks: 729 em-updates: 625


In [37]:
assert np.all(assets_weekly_iter["p_alpha"] > np.abs(assets_weekly_iter["p_beta"]))
assert np.all(assets_weekly_iter["p_delta"] > 0)
assets_weekly_iter[["p_alpha","p_beta","p_delta","p_mu"]].diff().describe()

,p_alpha,p_beta,p_delta,p_mu
count,728.000000,7.280000e+02,728.000000,7.280000e+02
mean,-0.000837,4.120923e-03,0.003519,7.525051e-07
std,0.012903,1.113627e-01,0.003330,1.704180e-04
min,-0.345287,-1.046451e-03,0.000000,-9.731436e-04
25%,-0.000283,-1.432111e-06,0.001426,-6.684920e-05
50%,-0.000054,-1.313765e-07,0.001895,0.000000e+00
75%,-0.000005,9.755716e-08,0.005603,6.208313e-05
max,0.000000,3.004722e+00,0.025178,2.270922e-03


*Important note*:
- The period 2012–2013 is used as an initial estimation sample to (i) construct a first asset path and (ii) obtain the first NIG parameter estimates via EM.
- From 2014 onward, parameters are updated on a weekly grid using a 2-year rolling window; PDs from this period are therefore “fully model-implied” (conditioned on prior 2 years of information).

### Key Idea

If log‑asset return over horizon \( \tau \) is NIG with cumulant

$$
\kappa(u;\tau)
=
\tau\!\left(
\mu u
+
\delta\!\left(
\sqrt{\alpha^{2}-\beta^{2}}
-
\sqrt{\alpha^{2}-(\beta+u)^{2}}
\right)
\right),
$$

then

$$
\ln A_{t+\tau}
=
\ln A_t + X_\tau,
\qquad
X_\tau \sim \text{NIG}(\alpha,\beta,\delta\tau,\mu\tau),
$$

and the terminal PD at horizon \( \tau \) is

$$
\text{PD}_t(\tau)
=
\Pr(A_{t+\tau} \le L_t)
=
\Pr\!\left(
X_\tau \le \ln\!\frac{L_t}{A_t}
\right)
=
F_{\text{NIG}}\!\left(
\ln\!\frac{L_t}{A_t};
\alpha,\beta,\delta\tau,\mu\tau
\right).
$$

We will compute this using:

```
scipy.stats.norminvgauss.cdf
```

**Assumption (important):**  
SciPy’s `norminvgauss(a, b, loc, scale)` matches your  
\((\alpha,\beta,\mu,\delta)\) as:

- `a = alpha`
- `b = beta`
- `loc = mu`
- `scale = delta`

This is the standard mapping used in most implementations.

In [ ]:
import numpy as np
from scipy.stats import norminvgauss

def pd_1y_terminal_nig(
    A_t: float,
    L_t: float,
    p: NIGParams,
    *,
    tau_steps: float,
) -> float:
    """
    Terminal PD over tau_steps (in the SAME time units as p was estimated on).

    If p is estimated on WEEKLY returns, use tau_steps=52 for 1 year.
    If p is estimated on DAILY returns,  use tau_steps=252 for 1 year.
    If p is estimated on MONTHLY returns, use tau_steps=12 for 1 year.
    """
    if (A_t <= 0) or (L_t <= 0):
        return np.nan

    p.validate()
    x = float(np.log(L_t / A_t))  # threshold on log-return

    # time scaling
    delta = float(p.delta) * tau_steps
    mu    = float(p.mu) * tau_steps
    alpha = float(p.alpha)
    beta  = float(p.beta)

    # SciPy mapping
    a = alpha * delta
    b = beta  * delta
    loc = mu
    scale = delta

    pd_val = norminvgauss.cdf(x, a=a, b=b, loc=loc, scale=scale)
    return float(np.clip(pd_val, 0.0, 1.0))


def month_end_calendar(g_panel: pd.DataFrame) -> pd.DatetimeIndex:
    d = g_panel[["date"]].drop_duplicates().sort_values("date")
    me = d.groupby(d["date"].dt.to_period("M"))["date"].max()
    return pd.DatetimeIndex(me.sort_values())


def pd_monthly_one_firm(
    g_panel: pd.DataFrame,
    *,
    gvkey: str,
    p0: NIGParams,
    p_cache: dict[pd.Timestamp, NIGParams],
    # inversion maturity (your option pricing maturity)
    tau_inv: float = 1.0,
    # PD horizon in "steps" matching EM frequency (52 if weekly params)
    tau_steps_pd: float = 52.0,
    U: float = 120.0,
    n: int = 2000,
    eta: float = 0.0,
) -> pd.DataFrame:

    g = g_panel.sort_values("date").copy()
    dates = month_end_calendar(g)

    out = []
    A_prev = None

    for d in dates:
        row = g.loc[g["date"] == d].iloc[0]
        E_obs = float(row["E"])
        L_t   = float(row["L"])
        r_t   = float(row["r"])

        p_t = pick_params_for_date(p_cache, pd.to_datetime(d), fallback=p0)

        A_hat, theta = invert_asset_one_date(
            E_obs, L_t, r_t, tau_inv, p_t,
            A_prev=A_prev, U=U, n=n,
        )

        pd_1y = pd_1y_terminal_nig(A_hat, L_t, p_t, tau_steps=tau_steps_pd)

        out.append({
            "gvkey": str(gvkey),
            "date": pd.to_datetime(d),
            "E": E_obs,
            "L": L_t,
            "r": r_t,
            "A_hat": float(A_hat),
            "theta": float(theta),
            "p_alpha": float(p_t.alpha),
            "p_beta": float(p_t.beta),
            "p_delta": float(p_t.delta),
            "p_mu": float(p_t.mu),
            "PD_1y": float(pd_1y),
        })

        A_prev = A_hat

    return pd.DataFrame(out).sort_values("date").reset_index(drop=True)


def pick_params_for_date(
    p_cache: dict[pd.Timestamp, NIGParams],
    d: pd.Timestamp,
    *,
    fallback: NIGParams,
) -> NIGParams:
    if not p_cache:
        return fallback

    d = pd.to_datetime(d)

    # p_cache keys might be Timestamp or datetime64; normalize
    keys = sorted(pd.to_datetime(list(p_cache.keys())))
    # pick last key <= d
    eligible = [k for k in keys if k <= d]
    if not eligible:
        return fallback

    return p_cache[eligible[-1]]

In [62]:
gv = "100022"
g_panel = nig_df[nig_df["gvkey"].astype(str) == gv].copy()

# p0 is your fallback pre-burn-in
p0 = NIGParams(alpha=15.0, beta=-3.0, delta=0.20, mu=0.00)

bmw_pd_m = pd_monthly_one_firm(
    g_panel=g_panel,
    gvkey="100022",
    p0=p0,
    p_cache=p_cache,
    tau_inv=1.0,        # your pricing inversion maturity
    tau_steps_pd=52.0,  # 1Y PD using weekly EM params
    U=120.0,
    n=2000,
)

# Apply evaluation start (burn-in complete): 2014-01-01
bmw_pd_eval = bmw_pd_m[bmw_pd_m["date"] >= "2014-01-01"].copy()

print(bmw_pd_m.head())
bmw_pd_eval["A_over_L"] = bmw_pd_eval["A_hat"] / bmw_pd_eval["L"]
print(bmw_pd_eval[["PD_1y","A_over_L"]].corr())
print(bmw_pd_eval["PD_1y"].describe())
print(bmw_pd_eval["A_over_L"].describe())

    gvkey       date             E             L         r         A_hat  \
0  100022 2012-01-31  3.936447e+10  8.576700e+10  0.002061  1.249548e+11   
1  100022 2012-02-29  4.179653e+10  8.576700e+10  0.001470  1.274375e+11   
2  100022 2012-03-30  4.059254e+10  9.632600e+10  0.001602  1.367644e+11   
3  100022 2012-04-30  4.322928e+10  9.632600e+10  0.001019  1.394572e+11   
4  100022 2012-05-31  3.678191e+10  9.632600e+10  0.000512  1.330586e+11   

      theta  p_alpha  p_beta  p_delta  p_mu     PD_1y  
0  2.654517     15.0    -3.0      0.2   0.0  0.980226  
1  2.610187     15.0    -3.0      0.2   0.0  0.979071  
2  2.620070     15.0    -3.0      0.2   0.0  0.981658  
3  2.576356     15.0    -3.0      0.2   0.0  0.980584  
4  2.538390     15.0    -3.0      0.2   0.0  0.983086  
             PD_1y  A_over_L
PD_1y     1.000000 -0.899992
A_over_L -0.899992  1.000000
count    144.000000
mean       0.450846
std        0.030476
min        0.341185
25%        0.445132
50%        0.462761
